## Задача 1:
Поступающий на работу плиточник заявляет, что за час выкладывает 4 кв. м. плитки. Мастер считает, что только 3 кв.м.

Через какое количество часов работы можно определить с вероятностью 95% кто из них прав? Какова будет при этом мощность проверки?

In [5]:
import math

# Заданные параметры
alpha = 0.05          # Уровень значимости (95%)
delta = 1             # Разница между скоростями (4 - 3 кв.м/час)
sigma = 0.5           # Стандартное отклонение (предположим, 0.5 кв.м/час)
beta = 0.2            # Мощность теста (1 - beta = 0.8)

# Значения квантилей нормального распределения (взятые из таблицы)
z_alpha = 1.96  # Квантиль для уровня значимости
z_beta_initial = 0.84   # Квантиль для мощности теста 80%

# Формула для нахождения необходимого количества часов работы
n = ((z_alpha + z_beta_initial) * sigma / delta) ** 2
n_hours = round(n, 2)

# Рассчитаем z_beta для фактического количества часов
z_beta = (math.sqrt(n) * delta / sigma) - z_alpha

# Рассчитаем мощность теста
# Используем аппроксимацию нормального распределения для мощности
beta_actual = 1 - (0.5 * (1 + math.erf(z_beta / math.sqrt(2))))
power = round(1 - beta_actual, 4)

# Выводим результаты
print(f"Необходимое количество часов для определения результата: {n_hours} часа(ов)")
print(f"Мощность проверки: {power}")


Необходимое количество часов для определения результата: 1.96 часа(ов)
Мощность проверки: 0.7995


## Задача 2:
Определите наилучшее полиномиальное приближение для процесса суточного изменения цены опциона (выберите любой опцион на любой бирже).

Рассчитайте p-значение для этого приближения и мощность проверки.

In [26]:
import requests
from bs4 import BeautifulSoup
import numpy as np
from numpy.linalg import solve as gauss
from statistics import mean
from math import sqrt, erf
from scipy import integrate

# Функция для загрузки данных
def GetData(data=[]):
    url = 'https://cryptocharts.ru/bitcoin-dollar/'
    response = requests.get(url)
    html_content = response.text

    soup = BeautifulSoup(html_content, 'html.parser')
    table_class = 'TableRates'
    table = soup.find('table', class_=table_class)

    rows = table.find_all('tr')
    for row in rows:
        cells = row.find_all('td')
        for cell in cells[1::2]:
            data.insert(0, cell.text.strip())

# Функция для нормального распределения
def f_norm(x, mu=0, sigma=1):
    return (1 + erf((x - mu) / (sqrt(2) * sigma))) / 2

# Функция для расчета коэффициентов полинома
def approx_poly(x, t, r):
    M = [[sum(ti ** (l + q) for ti in t) for q in range(r + 1)] for l in range(r + 1)]
    b = [sum(xi * ti ** l for xi, ti in zip(x, t)) for l in range(r + 1)]
    a = gauss(np.array(M, dtype="float64"), np.array(b, dtype="float64"))
    return a

# Функция для вычисления p-значения
def p_value(max_delta_x, sigma):
    integral, _ = integrate.quad(f_norm, float("-inf"), -max_delta_x, args=(0, sigma))
    return 2 * integral

# Функция для вычисления стандартного отклонения
def standard_deviation(x):
    return sqrt(sum((xi - mean(x)) ** 2 for xi in x) / (len(x) - 1))

# Функция для нахождения максимального значения по модулю
def find_max_by_abs(nums):
    return max(abs(num) for num in nums)

# Основная функция для тестирования
def test(x, n):
    alpha = 0.05
    dataset = x.copy()
    t = list(range(len(dataset)))
    
    # Аппроксимация
    A = approx_poly(dataset, t, n)
    x_approx = [sum(A[i] * ti ** i for i in range(len(A))) for ti in t]

    # Остатки
    epsilon = [x_approx[i] - dataset[i] for i in range(len(dataset))]
    mu = mean(epsilon)
    sigma = standard_deviation(epsilon)
    
    max_dev = find_max_by_abs([i - mu for i in epsilon])
    
    slow = -1.96 * sigma + mu
    shigh = 1.96 * sigma + mu
    
    return p_value(max_dev, sigma), x_approx, 1 - f_norm(shigh, mu, sigma) + f_norm(slow, mu, sigma)

def main():
    # Получение данных
    dataset = []
    GetData(dataset)
    dataset = [float(value.replace(',', '.')) for value in dataset]  # Преобразуем строки в числа

    max_p = -1
    max_n = 0
    max_w = 0
    
    for i in range(5, 50):
        p, x_appr, w = test(dataset, i)
        if max_p < p:
            max_p = p
            max_w = w
            max_n = i
    
    # Вывод результатов
    print("Результаты полиномиальной аппроксимации:")
    print(f"-----------------------------------------")
    print(f"Лучшее p_value: {max_p:.4f}")
    print(f"Мощность проверки: {max_w:.4f}")
    print(f"Степень полинома: {max_n}")
    print(f"-----------------------------------------")
    
if __name__ == "__main__":
    main()


Результаты полиномиальной аппроксимации:
-----------------------------------------
Лучшее p_value: 25.7889
Мощность проверки: 0.0500
Степень полинома: 49
-----------------------------------------


# Пояснительная записка

## 1. Импорт библиотек
```python
import requests
from bs4 import BeautifulSoup
import numpy as np
from numpy.linalg import solve as gauss
from statistics import mean
from math import sqrt, erf
from scipy import integrate

    requests: Используется для выполнения HTTP-запросов (например, для получения данных с веб-страницы).
    BeautifulSoup: Библиотека для парсинга HTML и извлечения данных из веб-страниц.
    numpy: Используется для численных расчетов, включая линейную алгебру.
    statistics: Включает функции для статистических расчетов, таких как среднее значение.
    math: Включает функции математического характера, такие как стандартное отклонение и функция ошибки.
    scipy: Библиотека для научных расчетов, используемая для численной интеграции.

2. Функция GetData()

python

def GetData(data=[]):
    url = 'https://cryptocharts.ru/bitcoin-dollar/'
    response = requests.get(url)
    html_content = response.text

    soup = BeautifulSoup(html_content, 'html.parser')
    table_class = 'TableRates'
    table = soup.find('table', class_=table_class)

    rows = table.find_all('tr')
    for row in rows:
        cells = row.find_all('td')
        for cell in cells[1::2]:
            data.insert(0, cell.text.strip())

    Эта функция загружает данные о ценах из указанного URL.
    requests.get(url): Получает HTML-код страницы.
    BeautifulSoup: Парсит HTML-код, чтобы найти таблицу с данными.
    data.insert(0, cell.text.strip()): Извлекает текст из ячеек таблицы и добавляет его в список data.

3. Функция f_norm()

python

def f_norm(x, mu=0, sigma=1):
    return (1 + erf((x - mu) / (sqrt(2) * sigma))) / 2

    Эта функция вычисляет значение нормального распределения для заданного x, mu (математическое ожидание) и sigma (стандартное отклонение).
    Используется функция ошибки erf, чтобы вычислить вероятность.

4. Функция approx_poly()

python

def approx_poly(x, t, r):
    M = [[sum(ti ** (l + q) for ti in t) for q in range(r + 1)] for l in range(r + 1)]
    b = [sum(xi * ti ** l for xi, ti in zip(x, t)) for l in range(r + 1)]
    a = gauss(np.array(M, dtype="float64"), np.array(b, dtype="float64"))
    return a

    Эта функция находит коэффициенты полинома степени r, который наилучшим образом аппроксимирует данные x.
    Использует метод наименьших квадратов для решения системы уравнений, основанной на матрице M и векторе b.

5. Функция p_value()

python

def p_value(max_delta_x, sigma):
    integral, _ = integrate.quad(f_norm, float("-inf"), -max_delta_x, args=(0, sigma))
    return 2 * integral

    Рассчитывает p-значение для максимального отклонения max_delta_x.
    integrate.quad(): Используется для численного интегрирования функции f_norm.

6. Функция standard_deviation()

python

def standard_deviation(x):
    return sqrt(sum((xi - mean(x)) ** 2 for xi in x) / (len(x) - 1))

    Вычисляет стандартное отклонение списка x.
    Использует формулу для расчета стандартного отклонения, основанную на средних значениях.

7. Функция find_max_by_abs()

python

def find_max_by_abs(nums):
    return max(abs(num) for num in nums)

    Находит максимальное по модулю значение в списке nums.

8. Основная функция test()

python

def test(x, n):
    alpha = 0.05
    dataset = x.copy()
    t = list(range(len(dataset)))
    
    # Аппроксимация
    A = approx_poly(dataset, t, n)
    x_approx = [sum(A[i] * ti ** i for i in range(len(A))) for ti in t]

    # Остатки
    epsilon = [x_approx[i] - dataset[i] for i in range(len(dataset))]
    mu = mean(epsilon)
    sigma = standard_deviation(epsilon)
    
    max_dev = find_max_by_abs([i - mu for i in epsilon])
    
    slow = -1.96 * sigma + mu
    shigh = 1.96 * sigma + mu
    
    return p_value(max_dev, sigma), x_approx, 1 - f_norm(shigh, mu, sigma) + f_norm(slow, mu, sigma)

    Основная логика для выполнения полиномиальной аппроксимации.
    Вычисляет остатки между фактическими значениями и аппроксимированными.
    Рассчитывает p-значение и мощность проверки.

9. Функция main()

python

def main():
    # Получение данных
    dataset = []
    GetData(dataset)
    dataset = [float(value.replace(',', '.')) for value in dataset]  # Преобразуем строки в числа

    max_p = -1
    max_n = 0
    max_w = 0
    
    for i in range(5, 50):
        p, x_appr, w = test(dataset, i)
        if max_p < p:
            max_p = p
            max_w = w
            max_n = i
    
    # Вывод результатов
    print("Результаты полиномиальной аппроксимации:")
    print(f"-----------------------------------------")
    print(f"Лучшее p_value: {max_p:.4f}")
    print(f"Мощность проверки: {max_w:.4f}")
    print(f"Степень полинома: {max_n}")
    print(f"-----------------------------------------")

    Главная функция, которая:
        Загружает данные.
        Преобразует их в числовой формат.
        Проводит полиномиальную аппроксимацию для разных степеней.
        Находит наилучшее p-значение, его мощность и степень полинома.
        Выводит результаты в читаемом формате.

Итог

Этот код позволяет получить полиномиальную аппроксимацию для данных о ценах опциона и вычислить статистические показатели (p-значение и мощность проверки). Результаты выводятся в удобочитаемом формате. Если у вас есть еще вопросы по конкретным частям или хотите более глубокого объяснения, дайте знать!